# **Automação Web e Busca de Informações com Python**

### **Desafio:**

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### **Instalar selenium:**

 - pip install selenium

### **Obtendo os dados**

In [1]:
#chromedriver --> Chrome
# geckodriver --> Firefox

from selenium import webdriver                  # Criar o navegador
from selenium.webdriver.common.by import By     # Localizar elementos (os itens de um site)
from selenium.webdriver.common.keys import Keys # Permite clicar teclas no teclado
import time
navegador = webdriver.Chrome()

# Passo 1: Entrar no Google
navegador.get('https://www.google.com/')


# Passo 2: Pesquisar a cotação do dólar
navegador.find_element(By.ID, 'APjFqb').send_keys('cotação dólar')
navegador.find_element(By.ID, 'APjFqb').send_keys(Keys.ENTER)

# Passo 3: Pegar a cotação do dólar
cotacao_dolar = navegador.find_element(By.XPATH, 
                                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(f'Cotação do dólar: {cotacao_dolar}')

# Passo 4: Pegar a cotação do euro
navegador.get('https://www.google.com/')

navegador.find_element(By.ID, 'APjFqb').send_keys('cotação euro')
navegador.find_element(By.ID, 'APjFqb').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, 
                                      '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(f'Cotação do euro: {cotacao_euro}')

# Passo 5: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(f'Cotação do ouro: {cotacao_ouro}')

# Fechar automaticamente o navegador quando terminar tudo
navegador.quit()

Cotação do dólar: 5.1506
Cotação do euro: 5.46040859
Cotação do ouro: 305.66


### **Agora vamos atualiza a nossa base de preços com as novas cotações**

- Importando a base de dados

In [2]:
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
# Passo 6: Atualizar a minha base de dados com as novas cotações
# Atualizar a cotação de acordo com a moeda correspondente

# dolar
# as linhas onde a coluna 'Moeda' == 'Dólar'
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)

# euro
# as linhas onde a coluna 'Moeda' == 'Euro'
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)

# ouro
# as linhas onde a coluna 'Moeda' == 'Ouro'
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# Atualizar preço de compra = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# Atualizar preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16336\2449052746.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.1506' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.150600,5150.548494,1.40,7210.767892
1,Carro Renault,4500.00,Euro,5.460409,24571.838655,2.00,49143.677310
2,Notebook Dell,899.99,Dólar,5.150600,4635.488494,1.70,7880.330440
3,IPhone,799.00,Dólar,5.150600,4115.329400,1.70,6996.059980
4,Carro Fiat,3000.00,Euro,5.460409,16381.225770,1.90,31124.328963
5,Celular Xiaomi,480.48,Dólar,5.150600,2474.760288,2.00,4949.520576
6,Joia 20g,20.00,Ouro,305.660000,6113.200000,1.15,7030.180000


### **Agora vamos exportar a nova base de preços atualizada**

In [4]:
tabela.to_excel('Produtos_Novos.xlsx', index=False)
# index=False --> Salva sem a coluna de índices